### Monika Belva Celesta - A12.2020.06392

#### 2. Topic Modelling Metode (Topik dari tiap sentimen yang didapatkan)
- Jelaskan metode yang digunakan (LDA/LSA/PLSA atau yang lain). Dijelaskan juga alasan memakai metode tsb.
-  Menjelaskan coding dan hasilnya

In [1]:
!pip install --upgrade gensim

  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.


  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1


In [2]:
!pip install pyldavis==3.2.1

In [3]:
!pip install Sastrawi

In [4]:
!pip install swifter

In [5]:
import pandas as pd
import numpy as np
import nltk

In [6]:
dataSB = pd.read_csv('PB_ROBOT.csv')

In [7]:
dataSB.head()

,username,bintang,review
0,salonayu10,5,Alhamdulillah paketnya sudah sampai dgn aman\n...
1,rubbyaqilla13,5,"Alhamdulillah, pesanan saya sudah sampai nih k..."
2,n*****1,2,"pertama kali aku kasih bintang di bwh 4, tpi e..."
3,erikatjhang,5,"Pengiriman sangat cepat, padahal belinya pas l..."
4,nandagiantari2002,5,"Jika ingin menghubungi layanan PLN 123, kamu d..."


In [8]:
dataSB = dataSB.drop(columns=['bintang'])
dataSB.head()

,username,review
0,salonayu10,Alhamdulillah paketnya sudah sampai dgn aman\n...
1,rubbyaqilla13,"Alhamdulillah, pesanan saya sudah sampai nih k..."
2,n*****1,"pertama kali aku kasih bintang di bwh 4, tpi e..."
3,erikatjhang,"Pengiriman sangat cepat, padahal belinya pas l..."
4,nandagiantari2002,"Jika ingin menghubungi layanan PLN 123, kamu d..."


In [9]:
# -------Case Folding-------
# gunakan fungsi Series.str.lower() pada Pandas

dataSB['review'] = dataSB['review'].str.lower()

print('Case Folding Result : \n')
print(dataSB['review'].head(5))

Case Folding Result : 

0    alhamdulillah paketnya sudah sampai dgn aman\n...
1    alhamdulillah, pesanan saya sudah sampai nih k...
2    pertama kali aku kasih bintang di bwh 4, tpi e...
3    pengiriman sangat cepat, padahal belinya pas l...
4    jika ingin menghubungi layanan pln 123, kamu d...
Name: review, dtype: object


In [10]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [11]:
dataSB['review'] = dataSB['review'].astype(str)

In [12]:
#------------ Tokenizing --------------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = str(text)
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\'," ")
    
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    
    # remove mention, link, hastag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    
    # remove incomplete URL
    return text.replace("https://", " ").replace("https://", " ")

dataSB['review'] = dataSB['review'].apply(remove_tweet_special)

# remove number
def remove_number(text):
    return re.sub(r"\d+", " ", text)

dataSB['review'] = dataSB['review'].apply(remove_number)

# remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataSB['review'] = dataSB['review'].apply(remove_punctuation)

# remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataSB['review'] = dataSB['review'].apply(remove_whitespace_LT)

# remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub("\s+",' ', text)

dataSB['review'] = dataSB['review'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub("\b[a-zA-Z]\b"," ", text)

dataSB['review'] = dataSB['review'].apply(remove_singl_char)

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataSB['textdata_tokens'] = dataSB['review'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(dataSB['textdata_tokens'].head())

[nltk_data] Downloading package punkt to C:\Users\ACER
[nltk_data]     SWIFT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizing Result : 

0    [alhamdulillah, paketnya, sudah, sampai, dgn, ...
1    [alhamdulillah, pesanan, saya, sudah, sampai, ...
2    [pertama, kali, aku, kasih, bintang, di, bwh, ...
3    [pengiriman, sangat, cepat, padahal, belinya, ...
4    [jika, ingin, menghubungi, layanan, pln, kamu,...
Name: textdata_tokens, dtype: object


In [13]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

dataSB['textdata_tokens_fdist'] = dataSB['textdata_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(dataSB['textdata_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(dan, 3), (aman, 2), (alhamdulillah, 1), (pak...
1    [(saya, 2), (sudah, 2), (sampai, 2), (dan, 2),...
2    [(yg, 3), (aku, 2), (di, 2), (dan, 2), (lampu,...
3    [(dan, 3), (dengan, 3), (aman, 2), (pengiriman...
4    [(telepon, 3), (layanan, 2), (pln, 2), (kamu, ...
Name: textdata_tokens_fdist, dtype: object


In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\ACER
[nltk_data]     SWIFT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords


# ------------- get stopword from NLTK stopword --------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ------------- manualy add stopword -------------------------------
# append additional stopword
list_stopwords.extend(["yg","dg","rt","dgn","ny","d","klo",
                      'kalo','amp','biar','bikin','bilang'
                      'gak','ga','krn','nya','nih','sih'
                      'si','tau','tdk','tuh','utk','ya'
                      'jd','jgn','sdh','aja','n','t'
                      'nyg','hehe','pen','u','nan','loh','rt',
                      '&amp','yah','bisnis','pandemi','indonesia'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

# remove stopwords pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['textdata_tokens_WSW'] = dataSB['textdata_tokens'].apply(stopwords_removal)

print(dataSB['textdata_tokens_WSW'].head())

0    [alhamdulillah, paketnya, aman, prodak, asli, ...
1    [alhamdulillah, pesanan, ka, sesuai, pesananpe...
2    [kali, kasih, bintang, bwh, tpi, emang, bener,...
3    [pengiriman, cepat, belinya, pas, event, packa...
4    [menghubungi, layanan, pln, saluran, telepon, ...
Name: textdata_tokens_WSW, dtype: object


In [16]:
# import file xlsx untuk proses normalisasi
!pip install gdown --id 139


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --id


In [17]:
normalizad_word = pd.read_excel('dataU.xlsx') #lokasi file

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]
        
def normalizad_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

dataSB['textdata_normalized'] = dataSB['textdata_tokens_WSW'].apply(normalizad_term)

dataSB['textdata_normalized'].head(10)

0    [alhamdulillah, paketnya, aman, prodak, asli, ...
1    [alhamdulillah, pesanan, ka, sesuai, pesananpe...
2    [kali, kasih, bintang, bwh, tpi, emang, bener,...
3    [pengiriman, cepat, belinya, pas, event, packa...
4    [menghubungi, layanan, pln, saluran, telepon, ...
5    [alhamdulillah, berfungsi, baikk, garansinya, ...
6    [produknya, bagus, beli, langsung, ofc, storen...
7    [maaf, diriview, packing, aman, produknya, bag...
8    [maaf, videonya, powerbanknya, bagus, cepat, t...
9    [sukaa, warnanya, semoga, bertahan, yaa, semog...
Name: textdata_normalized, dtype: object

In [18]:
# import Sastrawi package 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in dataSB['textdata_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))

4943


In [19]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    
    # untuk melihat hasilnya silahkan jalankan baris dibawah ini
    # print(term,":", term_dict[term])

In [20]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

dataSB['textdata_tokens_stemmed'] = dataSB['textdata_normalized'].swifter.apply(get_stemmed_term)

print(dataSB['textdata_tokens_stemmed'])

Pandas Apply:   0%|          | 0/3003 [00:00<?, ?it/s]

0       [alhamdulillah, paket, aman, prodak, asli, ori...
1       [alhamdulillah, pesan, ka, sesuai, pesananpeng...
2       [kali, kasih, bintang, bwh, tpi, emang, bener,...
3       [kirim, cepat, beli, pas, event, packagingnya,...
4       [hubung, layan, pln, salur, telepon, tau, hand...
                              ...                        
2998                           [kirim, cepat, moga, awet]
2999      [produk, oke, banget, slim, warna, hijau, mate]
3000                                                   []
3001    [powerbank, bagus, tidak, tebel, banget, tidak...
3002    [paham, erti, bioteknologi, istilah, muncul, a...
Name: textdata_tokens_stemmed, Length: 3003, dtype: object


In [21]:
#stopwords #2

from nltk.corpus  import stopwords

#.......................get stopworods from nLTK stopwords......................
#get stopwords indonesia
list_stopwords = stopwords.words('indonesian')

#.......................manualy add stopword....................................
#append additional stopwords 
list_stopwords.extend(["sampai","dengan","rapih","terimakasih","seller"])

#convert list to dictionary 
list_stopwords = set (list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['textdata_tokens_stemmed2'] = dataSB ['textdata_tokens_stemmed'].apply (stopwords_removal)

print(dataSB['textdata_tokens_stemmed2'].head())

0    [alhamdulillah, paket, aman, prodak, asli, ori...
1    [alhamdulillah, pesan, ka, sesuai, pesananpeng...
2    [kali, kasih, bintang, bwh, tpi, emang, bener,...
3    [kirim, cepat, beli, pas, event, packagingnya,...
4    [hubung, layan, pln, salur, telepon, tau, hand...
Name: textdata_tokens_stemmed2, dtype: object


In [22]:
for i in range(len(dataSB)):
    a = dataSB.iloc[i][6]
    document.append(a)
    
document[0:5]

['paham', 'pengertian', 'bioteknologi', 'istilah', 'muncul']

In [23]:
doc_clean = dataSB['textdata_tokens_stemmed2']
doc_clean

0       [alhamdulillah, paket, aman, prodak, asli, ori...
1       [alhamdulillah, pesan, ka, sesuai, pesananpeng...
2       [kali, kasih, bintang, bwh, tpi, emang, bener,...
3       [kirim, cepat, beli, pas, event, packagingnya,...
4       [hubung, layan, pln, salur, telepon, tau, hand...
                              ...                        
2998                           [kirim, cepat, moga, awet]
2999      [produk, oke, banget, slim, warna, hijau, mate]
3000                                                   []
3001    [powerbank, bagus, tebel, banget, ngehabisin, ...
3002    [paham, erti, bioteknologi, istilah, muncul, a...
Name: textdata_tokens_stemmed2, Length: 3003, dtype: object

In [41]:
!pip install numpy==1.24.1

## LDA

In [42]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<4420 unique tokens: ['alhamdulillah', 'aman', 'an', 'asli', 'awer']...>


In [43]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik

In [44]:
# Running and Training LDA model on the document term matrix
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes = 50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.020*"bgt" + 0.018*"warna" + 0.008*"pas" + 0.007*"kirim" + 0.007*"diskon" + 0.006*"dapet" + 0.006*"sale" + 0.006*"tp" + 0.006*"putih" + 0.005*"bangett"'),
 (1,
  '0.061*"moga" + 0.054*"awet" + 0.038*"bagus" + 0.027*"banget" + 0.023*"barang" + 0.017*"coba" + 0.015*"warna" + 0.013*"fungsi" + 0.013*"ya" + 0.013*"kirim"'),
 (2,
  '0.067*"barang" + 0.058*"kirim" + 0.053*"produk" + 0.050*"cepat" + 0.031*"sesuai" + 0.029*"bagus" + 0.024*"kualitas" + 0.021*"aman" + 0.020*"original" + 0.020*"harga"')]

In [45]:
# word Count of Topic Keyword

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i, weight, counter[word]])
        
df_imp_wcount = pd.DataFrame(out, columns = ['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

        word  topic_id  importance  word_count
0        bgt         0    0.019701         157
1      warna         0    0.017972         357
2        pas         0    0.008067         134
3      kirim         0    0.006665         935
4     diskon         0    0.006569          42
5      dapet         0    0.005990          79
6       sale         0    0.005947          52
7         tp         0    0.005821          89
8      putih         0    0.005812          41
9    bangett         0    0.005499          32
10      moga         1    0.060799         856
11      awet         1    0.053893         771
12     bagus         1    0.038019         927
13    banget         1    0.026714         415
14    barang         1    0.022532        1157
15      coba         1    0.017139         260
16     warna         1    0.015028         357
17    fungsi         1    0.013088         302
18        ya         1    0.012542         230
19     kirim         1    0.012516         935
20    barang 

In [46]:
# simpan 
df_imp_wcount.to_csv('id_imp_wcount.csv')

In [47]:
# Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, text=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldmodel.per_word_topics else row_list 
        #print(row)
        row = sorted(row,key=lambda x:(x[1]), reverse=True)
        # Get the dominant topic, perc contribution and keywords for each document 
        for j, (topic_num, prop_topic) in enumerate (row):
            if j ==0: # => dominant topic 
                wp = ldmodel.show_topic (topic_num)
                topic_keywords = ", ".join ([word for word, prop in wp])
                sent_topics_df = sent_topics_df. append(pd.series([int(topic_num), round (prop_topic,4), topic_keywords]), ignore_index=true)
            else:
                break
    sent_topics_df.colums=['Dominant_topics','Perc_Contribution', 'Topic_keywords']

    # add original text to the end of the output 
    contents=pd.series (texts)
    sent_topic_df= pd.concat ([sent_topic_df, contents], axis=1)
    return(sent_topics_df)

In [48]:
df_topics_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywwords.reset_index()
df_dominant_topic.column = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

TypeError: format_topics_sentences() got an unexpected keyword argument 'texts'

In [49]:
# simpan 
df_dominant_topic.to_csv('df_dominant_topic.csv')

NameError: name 'df_dominant_topic' is not defined

In [50]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topic
pyLDAvis.enable_notebook()

In [51]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

In [52]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [53]:
# proses ini mungki agak lama
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [54]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [55]:
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+str(total_topics) + '.html')

In [40]:
# proses ini mungkin agak lama
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.129393  0.128556       1        1  46.037099
2     -0.104647 -0.137949       2        1  41.010230
1      0.234040  0.009393       3        1  12.952671, topic_info=          Term        Freq       Total Category  logprob  loglift
106     produk  617.000000  617.000000  Default  30.0000  30.0000
112       moga  796.000000  796.000000  Default  29.0000  29.0000
24        awet  705.000000  705.000000  Default  28.0000  28.0000
49       harga  297.000000  297.000000  Default  27.0000  27.0000
141   kualitas  284.000000  284.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
9       fungsi   14.775755  264.018561   Topic3  -5.5592  -0.8392
82        suka   13.287629  148.179039   Topic3  -5.6654  -0.3677
572         gk    9.940048   35.027905   Topic3  -5.9557   0.7843
78   powerbank   10.758502  125.192807   Topic3  -5.8765  -0.4103
5       banget   10.489199  365.189420   Topic3  -5.9019  -1.5062

[188 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
868       1  0.948772         aamiin
0         1  0.994220  alhamdulillah
0         2  0.005403  alhamdulillah
1         1  0.671847           aman
1         2  0.328391           aman
...     ...       ...            ...
595       2  0.027773           yaaa
998       2  0.066365          yaaaa
998       3  0.929114          yaaaa
98        3  0.990688            you
161       3  0.919520           youu

[216 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])